<a href="https://colab.research.google.com/github/annasajkh/GPT-2-Music/blob/main/GPT2_Music_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cp drive/MyDrive/checkpoint_run1.tar /content/
!tar -xf checkpoint_run1.tar

In [ ]:
!pip install music21

In [ ]:
%cp drive/MyDrive/adl-piano-midi.zip /content/

In [ ]:
!unzip adl-piano-midi.zip

In [ ]:
#code from https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5

from music21 import converter, instrument, note, chord, stream

def midi2notes(path):
  notes = []
  midi = converter.parse(path)
  notes_to_parse = None

  parts = instrument.partitionByInstrument(midi)   
  
  if parts:
    notes_to_parse = parts.parts[0].recurse()
  else:
    notes_to_parse = midi.flat.notes
  
  count = 0
  
  for element in notes_to_parse:
    if isinstance(element, note.Note):
      notes.append(str(element.pitch))
    elif isinstance(element, chord.Chord):
      notes.append(".".join(str(n) for n in element.normalOrder))
  
  return notes

def notes2midi(notes, output_path):
  offset = 0
  output_notes = []

  for pattern in notes:
    if ("." in pattern) or pattern.isdigit():
      notes_in_chord = pattern.split(".")
      notes = []
      for current_note in notes_in_chord:
          new_note = note.Note(int(current_note))
          new_note.storedInstrument = instrument.Piano()
          notes.append(new_note)
      new_chord = chord.Chord(notes)
      new_chord.offset = offset
      output_notes.append(new_chord)
    else:
      new_note = note.Note(pattern)
      new_note.offset = offset
      new_note.storedInstrument = instrument.Piano()
      output_notes.append(new_note)
    offset += 0.5
  
  midi_stream = stream.Stream(output_notes)
  midi_stream.write("midi", fp=output_path)

In [ ]:
import os
from tqdm import tqdm
count = 0

f = open("drive/MyDrive/piano_music_dataset.txt", "a")

for root, dirs, files in os.walk("/content/adl-piano-midi"):
  for file in files:
    if file.lower().endswith(".mid"):
      try:
        path = os.path.join(root, file)

        notes = midi2notes(path)

        
        print(count)
        count += 1

        text = ",".join(notes)
        f.write("<|startoftext|>" + text + "<|endoftext|>\n")
      except Exception as e:
        print(e)
f.close()

In [ ]:
# with open("drive/MyDrive/game_music_dataset.txt", "r") as f:
#   text = f.read().replace("\n<|endoftext|>\n","<|endoftext|>").replace("<|startoftext|>\n", "").split("<|endoftext|>")
#   print(print([len(i) for i in text]))
#   print(len(text))
#   text = "<|endoftext|>".join(text)
#   with open("drive/MyDrive/game_music_dataset_1.txt", "w") as f:
#     f.write(text)

In [ ]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple

from os import path
import gpt_2_simple as gpt2
import tensorflow as tf

%cd drive/MyDrive/GPT-2_Music/

if not path.exists("models"):
  gpt2.download_gpt2(model_name="355M")

action = "generate"


if action == "train":
  while True:
    tf.reset_default_graph()
    session = gpt2.start_tf_sess()

    gpt2.finetune(session,
                  dataset="../piano_music_dataset.txt",
                  model_name='355M',
                  steps=100_000,
                  run_name='run1',
                  print_every=10,
                  sample_every=100_000,
                  save_every=100
                  )
if action == "generate":
  tf.reset_default_graph()
  session = gpt2.start_tf_sess()
  gpt2.load_gpt2(session)

  while True:
    text = ""
    while len(text) < 3:
      text = gpt2.generate(session,
                    temperature=0.7,
                    batch_size=1,
                    prefix="<|startoftext|>",
                    truncate="<|endoftext|>",
                    include_prefix=False,
                    length=1024,
                    return_as_list=True
                    )[0]
    print(text)



In [ ]:
gpt2.copy_checkpoint_to_gdrive("run1")

In [ ]:
cd ../../../